In [2]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd

boston = load_boston()
dfX = pd.DataFrame(boston.data, columns= boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df= pd.concat([dfX, dfy], axis=1)

N=len(df)
ratio=0.7
np.random.seed(0)
idx_train = np.random.choice(np.arange(N), np.int(ratio*N))
idx_test = list(set(np.arange(N)).difference(idx_train))

df_train = df.iloc[idx_train]
df_test = df.iloc[idx_test]

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_33952/3675737880.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  idx_train = np.random.choice(np.arange(N), np.int(ratio*N))


In [3]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [5]:
import statsmodels.api as sm
model = sm.OLS.from_formula("MEDV~" + "+".join(boston.feature_names), data=df_train)
result= model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     102.2
Date:                Sun, 30 Apr 2023   Prob (F-statistic):          9.64e-117
Time:                        13:42:02   Log-Likelihood:                -1171.5
No. Observations:                 404   AIC:                             2371.
Df Residuals:                     390   BIC:                             2427.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.0917      5.522      6.898      0.0

In [6]:
#y_pred와 y_test 비교
pred = result.predict(df_test)

rss= ((df_test.MEDV-pred)**2).sum()
tss= ((df_test.MEDV- df_test.MEDV.mean())**2).sum()
rsquared = 1-rss/tss
rsquared

0.5892223849182501

### 해결해야할 문제 
1. 과적합문제 (overfitting)
2. 다중공선성 문제 (multicollineary)

### 1. Overfitting 확인하기 - Cross Validation


In [7]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)
df_train.shape, df_test.shape

((354, 14), (152, 14))

In [8]:
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(dfX, dfy, test_size=0.3, random_state=0)

In [9]:
from sklearn.model_selection import KFold

scores=np.zeros(5)
cv=KFold(5, shuffle=True,random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train= df.iloc[idx_train]
    df_test=df.iloc[idx_test]
    
    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
    result=model.fit()
    
    pred=result.predict(df_test)
    rss = ((df_test.MEDV - pred) ** 2).sum()
    tss = ((df_test.MEDV - df_test.MEDV.mean()) ** 2).sum()
    rsquared=1 - rss/tss
    scores[i]=rsquared
    print("Train R2={:.8f}, Validation R2= {:.8f}".format(result.rsquared, rsquared))

Train R2=0.77301356, Validation R2= 0.58922238
Train R2=0.72917058, Validation R2= 0.77799144
Train R2=0.74897081, Validation R2= 0.66791979
Train R2=0.75658611, Validation R2= 0.66801630
Train R2=0.70497483, Validation R2= 0.83953317


In [10]:
from sklearn.model_selection import KFold

scores=np.zeros(5)
cv=KFold(5, shuffle=True,random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train= df.iloc[idx_train]
    df_test=df.iloc[idx_test]
    
    model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df_train)
    result=model.fit()
    
    pred=result.predict(df_test)
    diff= np.log(df_test.MEDV+1) - np.log(pred+1)
    mean_error = np.square(diff).mean()
    RMSLE = np.sqrt(mean_error)
    scores[i] = RMSLE
    print("Train RMSLE={:.8f}, Validation RMSLE= {:.8f}".format(result.RMSLE, RMSLE))

AttributeError: 'OLSResults' object has no attribute 'RMSLE'

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


## log 필드 추가, Model Evaluation 함수 설정하기

In [15]:
df['MEDV_log'] = np.log(df['MEDV'] + 1)

In [14]:
(506/3)*2

337.3333333333333

In [17]:
def split_df(df):
    return df[:337], df[337:]

In [31]:
def predict_price(model, select_columns):
    df_train, df_test = split_df(df)

    X_train = df_train[select_columns]
    y_train = df_train['MEDV_log']
    X_test = df_test[select_columns]
    y_test= df_test['MEDV_log']
    
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = np.exp(y_pred) - 1

    return y_pred

In [32]:
from sklearn.metrics import make_scorer

def get_rmsle(y_actual, y_pred):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

In [33]:
rmsle_scorer = make_scorer(get_rmsle, greater_is_better=False)

In [34]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV', 'MEDV_log'],
      dtype='object')

In [35]:
df_train, df_test = split_df(df)
#ml_columns에는 x만 포함
ml_columns = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'
]
X_train = df_train[ml_columns].copy()
y_train = df_train['MEDV']
rmsle_scorer = make_scorer(get_rmsle, greater_is_better=False)

In [36]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

hyperparams = {'max_iter': [7000, 10000, 20000, 30000], 
               'alpha': 1/np.array([0.1, 1, 2, 3, 4, 10, 30,2000,5000, 70000])
}

lasso_grid=GridSearchCV(estimator = Lasso(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'alpha': 0.25, 'max_iter': 7000}


In [37]:
lasso_model = lasso_grid.best_estimator_
ml_pred = predict_price(lasso_model, ml_columns)

In [42]:
y_test= df_test['MEDV']

In [43]:
diff = np.log(ml_pred + 1) - np.log(y_test + 1)
mean_error = np.square(diff).mean()
RMLSE = np.sqrt(mean_error)
print(RMLSE)

0.2909305205062572
